## Indox Retrieval Augmentation
Here, we will explore how to work with Indox Retrieval Augmentation. We are using Mistral as LLM model and HuggingFace for our embedding, we should set our HUGGINGFACE_API_KEY and MISTRAL_API_KEY as an environment variable.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/cookbook/indoxRag/mistral_clusteredSplit.ipynb)

In [ ]:
!pip install indoxRag
!pip install mistralai
!pip install chromadb

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
  python -m venv indox
```

2. **Activate the virtual environment:**
```bash
  indox\Scripts\activate
```


### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
   
2. **Activate the virtual environment:**
```bash
  source indox/bin/activate
```

### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
  pip install -r requirements.txt
```


In [ ]:
!wget https://raw.githubusercontent.com/osllmai/inDox/master/Demo/sample.txt

## Setting Up the Python Environment

If you are running this project in your local IDE, please create a Python environment to ensure all dependencies are correctly managed. You can follow the steps below to set up a virtual environment named `indox`:

### Windows

1. **Create the virtual environment:**
```bash
python -m venv indox
```
2. **Activate the virtual environment:**
```bash
indox_judge\Scripts\activate
```

### macOS/Linux

1. **Create the virtual environment:**
   ```bash
   python3 -m venv indox
```

2. **Activate the virtual environment:**
    ```bash
   source indox/bin/activate
```
### Install Dependencies

Once the virtual environment is activated, install the required dependencies by running:

```bash
pip install -r requirements.txt
```


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
MISTRAL_API_KEY = os.getenv('MISTRAL_API_KEY')
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')

### Creating an instance of IndoxTetrivalAugmentation

To effectively utilize the Indox Retrieval Augmentation capabilities, you must first create an instance of the IndoxRetrievalAugmentation class. This instance will allow you to access the methods and properties defined within the class, enabling the augmentation and retrieval functionalities.

In [ ]:
from indoxRag import IndoxRetrievalAugmentation
indox = IndoxRetrievalAugmentation()

INFO: IndoxRetrievalAugmentation initialized

            ██  ███    ██  ██████   ██████  ██       ██
            ██  ████   ██  ██   ██ ██    ██   ██  ██
            ██  ██ ██  ██  ██   ██ ██    ██     ██
            ██  ██  ██ ██  ██   ██ ██    ██   ██   ██
            ██  ██  █████  ██████   ██████  ██       ██
            


In [3]:
indox.__version__

'0.1.15'

### Generating response using Mistral's language models 
MistralQA class is used to handle question-answering task using Mistral's language models from HuggingFace. This instance creates HuggingFaceEmbedding class to specifying embedding model.By using UnstructuredLoadAndSplit function we can import various file types and split them into chunks.

In [ ]:
from indoxRag.llms import Mistral
from indoxRag.embeddings import HuggingFaceEmbedding
from indoxRag.data_loader_splitter import ClusteredSplit
from indoxRag.embeddings import MistralEmbedding
mistral_qa = Mistral(api_key=MISTRAL_API_KEY)
# embed_hf = HuggingFaceEmbedding(model="multi-qa-mpnet-base-cos-v1")
embed_mistral = MistralEmbedding(MISTRAL_API_KEY,model="mistral-embed")
file_path = "sample.txt"



INFO: Initializing MistralAI with model: mistral-medium-latest
INFO: MistralAI initialized successfully
INFO: Initialized MistralEmbedding with model: mistral-embed


In [8]:
loader_splitter = ClusteredSplit(file_path=file_path,summary_model=mistral_qa,embeddings=embed_mistral)
docs = loader_splitter.load_and_chunk()

INFO: ClusteredSplit initialized successfully
INFO: Starting processing for documents
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: mistral-embed
INFO: --Generated 6 clusters--
INFO: Generating summary for documentation
INFO: Generating summary for documentation
INFO: Generating summary for documentation
INFO: Generating summary for documentation
INFO: Generating summary for documentation
INFO: Generating summary for documentation
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: mistral-embed
INFO: --Generated 1 clusters--
INFO: Generating summary for documentation
INFO: Completed chunking & clustering process
INFO: Successfully obtained all documents


 Here ChromaVectorStore handles the storage and retrieval of vector embeddings by specifying a collection name and sets up a vector store where text embeddings can be stored and queried.

In [ ]:
from indoxRag.vector_stores import Chroma
db = Chroma(collection_name="sample",embedding_function=embed_mistral)

INFO: Connection to the vector store database established successfully


### load and preprocess data
This part of code demonstrates how to load and preprocess text data from a file, split it into chunks, and store these chunks in the vector store that was set up previously.

In [ ]:
from indoxRag.data_loader_splitter import UnstructuredLoadAndSplit
loader_splitter = UnstructuredLoadAndSplit(file_path=file_path,max_chunk_size=400)
docs = loader_splitter.load_and_chunk()

INFO: UnstructuredLoadAndSplit initialized successfully
INFO: Getting all documents
INFO: Starting processing
INFO: Using title-based chunking
INFO: Completed chunking process
INFO: Successfully obtained all documents


In [11]:
len(docs)

40

In [12]:
db.add(docs=docs)

INFO: Storing documents in the vector store
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: mistral-embed
INFO: Document added successfully to the vector store.
INFO: Documents stored successfully


### Retrieve relevant information and generate an answer
The main purpose of these lines is to perform a query on the vector store to retrieve the most relevant information (top_k=5) and generate an answer using the language model.

In [13]:
query = "How cinderella reach her happy ending?"
retriever = indox.QuestionAnswer(vector_database=db,llm=mistral_qa,top_k=5)

invoke(query) method sends the query to the retriever, which searches the vector store for relevant text chunks and uses the language model to generate a response based on the retrieved information.
Context property retrieves the context or the detailed information that the retriever used to generate the answer to the query. It provides insight into how the query was answered by showing the relevant text chunks and any additional information used.

In [14]:
retriever.invoke(query)

INFO: Retrieving context and scores from the vector database
INFO: Embedding documents
INFO: Starting to fetch embeddings for texts using engine: mistral-embed
INFO: Generating answer without document relevancy filter
INFO: Answering question
INFO: Attempting to generate an answer for the question
INFO: Query answered successfully


"Cinderella's happy ending began when her fairy godmother appeared and granted her wishes. With the help of the fairy godmother, Cinderella was able to attend the king's festival and dance with the prince. Despite the stepsisters' attempts to keep her from attending, Cinderella was able to go to the ball with the help of her magical dress, glass slippers, and carriage.\n\nAt the ball, the prince fell in love with Cinderella and danced with her all night. However, when the clock struck midnight, Cinderella had to leave in a hurry and accidentally left behind one of her glass slippers. The prince, determined to find the woman he had fallen in love with, searched the entire kingdom for the owner of the glass slipper. When he finally found Cinderella and the slipper fit her foot, he knew he had found his true love.\n\nIn the end, Cinderella married the prince and lived happily ever after. Despite the hardships she faced with her evil stepmother and stepsisters, Cinderella was able to overc

In [18]:
retriever.context

['by the hearth in the cinders. And as on that account she alwayslooked dusty and dirty, they called her cinderella.It happened that the father was once going to the fair, and heasked his two step-daughters what he should bring back for them.Beautiful dresses, said one, pearls and jewels, said the second.And you, cinderella, said he, what will you have. Father',
 "to appear among the number, they were delighted, called cinderellaand said, comb our hair for us, brush our shoes and fasten ourbuckles, for we are going to the wedding at the king's palace.Cinderella obeyed, but wept, because she too would have liked togo with them to the dance, and begged her step-mother to allowher to do so. You go, cinderella, said she, covered in dust and",
 'cinderella expressed a wish, the bird threw down to her what shehad wished for.It happened, however, that the king gave orders for a festivalwhich was to last three days, and to which all the beautiful younggirls in the country were invited, in orde

### With AgenticRag

AgenticRag stands for Agentic Retrieval-Augmented Generation. This concept combines retrieval-based methods and generation-based methods in natural language processing (NLP). The key idea is to enhance the generative capabilities of a language model by incorporating relevant information retrieved from a database or a vector store. 
 AgenticRag is designed to provide more contextually rich and accurate responses by utilizing external knowledge sources. It retrieves relevant pieces of information (chunks) from a vector store based on a query and then uses a language model to generate a comprehensive response that incorporates this retrieved information.

In [20]:
agent = indox.AgenticRag(llm=mistral_qa,vector_database=db,top_k=5)
agent.run("where does messi plays right now?")

2024-07-09 19:30:22,358 INFO:HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
2024-07-09 19:30:23,247 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 19:30:29,288 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 19:30:30,881 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 19:30:32,840 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-09 19:30:33,450 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


INFO: No Relevant document found, Start web search
INFO: No Relevant Context Found, Start Searching On Web...
INFO: Answer Base On Web Search
INFO: Answering question
INFO: Attempting to generate an answer for the question


2024-07-09 19:30:39,215 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


INFO: Check For Hallucination In Generated Answer Base On Web Search


2024-07-09 19:30:39,726 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


INFO: Hallucination detected, Regenerate the answer...
INFO: Answering question
INFO: Attempting to generate an answer for the question


2024-07-09 19:30:40,687 INFO:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


'Based on the provided context information, Lionel Messi currently plays for Inter Miami CF in Major League Soccer (MLS).'